In [3]:
from ast import literal_eval as make_tuple
from scipy import ndimage
import numpy as np
from skimage.transform import rescale, resize, downscale_local_mean

"""
when photons are absorbed in silicon sensors and turns into electrons, there is a conversion rate.
Every 3.6 eV turns into 1 electron. For example, 360 eV photon will turn into 100 electrons and 7.2 keV
photon will turn into about 2000 electrons. The process of course is not fully deterministic and there
are many other sources of uncertainties and electron/charge loss mechanisms.
"""

def makeSpeckle(size, speckleSize = 50): # 30 will lead to a factof of 3 over sampling, thus better maintain the contrast
    image = np.zeros(size)
    xPhasors = int(np.round(size[0]/speckleSize))
    yPhasors = int(np.round(size[1]/speckleSize))
    rndPhasor = np.zeros(size)
    rndPhasor[0:xPhasors,0:yPhasors] = np.exp(np.random.random([xPhasors,yPhasors])*2.j*np.pi)
    speckleField = np.fft.fft2(rndPhasor)
    speckleIntensity = np.abs(speckleField)**2
    return speckleIntensity/np.mean(speckleIntensity.flatten())


def AddShotNoise(speckle, kbar):
    """
    generate speckle pattern as discrete photons by introducing shot noise.
    variable kbar is the average photon density over the field of view.
    """
    speckle = speckle/np.mean(speckle.flatten())*kbar;
    return np.random.poisson(speckle)

def ApplyChargeCloud(photonPattern, cloudSize, photonEnergy):
    """
    this step blurs the photon map by the charge cloud size, assuming
    every electron in silicon takes 3.6 eV to generate.
    """
    return ndimage.filters.gaussian_filter(photonPattern*photonEnergy/3.6, cloudSize/2.35, mode='wrap', truncate=7)

def rebin(a, shape):
    """
    pattern rebinning down to smaller sizes
    by adding MXN blocks together. IDL type, pretty quick.
    """
    sh = shape[0],a.shape[0]//shape[0],shape[1],a.shape[1]//shape[1]
    return a.reshape(sh).sum(-1).sum(1)
            

def digitizeCharge(chargePattern, chargePerADU=15, gainMap=None):
    """
    digitize the charge pattern to ADUs, with gain variation
    """
    if gainMap is None:
        gainMap = np.random.randn(chargePattern.shape[0], chargePattern.shape[1])*0.05+1;
    return chargePattern/chargePerADU # Should it bet [a*b for a,b in zip(chargePattern/chargePerADU,gainMap)]

    
def AddReadoutNoise(chargePattern, pixelReadoutNoise):
    """
    add readout noise to the detector.
    """
    return chargePattern + np.random.randn(chargePattern.shape[0], chargePattern.shape[1])*pixelReadoutNoise

In [4]:
def makeWeakSpeckle(speckle, kbar = 0.1, speckleSize = 50, gridSize=[1024,1024], detectorSize=[128,128], 
                    chargeCloudSize=2, photonEnergy=9500, readoutNoise=10):
    """
    use the functions in the earlier section to produce a single detector image of
    a weak speckle pattern with average signal rate of kbar.
    """
    kbar_grid = kbar/gridSize[0]/gridSize[1]*detectorSize[0]*detectorSize[1]
    chargeCloudSize_grid = chargeCloudSize*gridSize[0]/detectorSize[0]
    #speckle = makeSpeckle([1024,1024],50)
    #print(chargeCloudSize_grid)
    speckleN = AddShotNoise(speckle, kbar_grid)
    speckleNE = ApplyChargeCloud(speckleN, chargeCloudSize_grid, photonEnergy)
    speckleD = rebin(speckleNE, detectorSize)
    speckleDD = digitizeCharge(speckleD)
    return AddReadoutNoise(speckleDD, readoutNoise)

def makeFlatPoisson(kbar, gridSize=[1024,1024], detectorSize =[128,128], chargeCloudSize=2.0, photonEnergy=9500.0,
                    readoutNoise=10.0):
    
    chargeCloudSize_grid = chargeCloudSize*gridSize[0]/detectorSize[0]
    kbar_grid = kbar/gridSize[0]/gridSize[1]*detectorSize[0]*detectorSize[1]
    pattern = np.ones(gridSize)
    patternN = AddShotNoise(pattern, kbar_grid)
    patternNE = ApplyChargeCloud(patternN, chargeCloudSize_grid, photonEnergy)
    patternD = rebin(patternNE, detectorSize)
    patternDD = digitizeCharge(patternD)
    return AddReadoutNoise(patternDD, readoutNoise)

In [33]:
import multiprocessing as mp
from numba import prange
from time import time

def nameBatcher(kbar,gridSize,speckleSize,detectorSize,chargeCloudSize,photonEnergy,readoutNoise):
    return "k" + str(kbar) + "gs" + str(gridSize[0]) + "ss" + str(speckleSize) + "ds" + str(detectorSize[0]) + \
            "ccs"+ str(chargeCloudSize) + "pE" + str(photonEnergy) + "rn" + str(readoutNoise)
    
def DataBatcher(kbarBatch,gridSizeBatch,speckleSizeBatch,detectorSizeBatch,
                chargeCloudSizeBatch,photonEnergyBatch,readoutNoiseBatch,N):
    '''
    Returns the data in batches for multithreading and the name of the file that will be saved.
    '''
    dataBatch = []
    nameBatch = []
    for kbar in kbarBatch:
        for gridSize in gridSizeBatch:
            for speckleSize in speckleSizeBatch:
                for detectorSize in detectorSizeBatch:
                    for chargeCloudSize in chargeCloudSizeBatch:
                        for photonEnergy in photonEnergyBatch:
                            for readoutNoise in readoutNoiseBatch:
                                dataBatch.append((kbar,gridSize,speckleSize,detectorSize,
                                                  chargeCloudSize,photonEnergy,readoutNoise,N))
                                nameBatch.append(nameBatcher(kbar,gridSize,speckleSize,detectorSize,
                                                             chargeCloudSize,photonEnergy,readoutNoise))
    
    return dataBatch, nameBatch

def DetectorImageBatch(kbar,gridSize,speckleSize,detectorSize,chargeCloudSize,photonEnergy,readoutNoise,N):
    """generate N detector images"""
    tic = time()
    images = []
    speckle = makeSpeckle(size = gridSize,speckleSize = speckleSize)
    
    np.random.seed() # Otherwise each thread gets the same pseudorandom number and you get the same plot 4 times

    for i in prange(N):
        if np.mod(i,10) == 0:
            print(i)
        detectorOutput = makeWeakSpeckle(speckle,kbar,speckleSize,gridSize,detectorSize,chargeCloudSize,photonEnergy,readoutNoise)
        images.append(detectorOutput)
    toc = time()
    print(toc-tic)
    return images

# Note: after using threading multiple times, there is usually an error that says
# that there are too many instances open. Restart kernel and reset output to continue.
def threading():
    pool = mp.Pool(4)
    N = 10000
    
    kbarBatch = [0.1]
    gridSizeBatch = [[2**10,2**10]]
    speckleSizeBatch = [50]
    detectorSizeBatch = [[32,32]]
    chargeCloudSizeBatch = [0.5]
    photonEnergyBatch = [9500]
    readoutNoiseBatch = [3]
    data,names = DataBatcher(kbarBatch,gridSizeBatch,speckleSizeBatch,detectorSizeBatch,
                chargeCloudSizeBatch,photonEnergyBatch,readoutNoiseBatch,N)
    
    result = pool.starmap(DetectorImageBatch, data)
    return result,names

def threadImages((kbar,gridSize,speckleSize,detectorSize,chargeCloudSize,photonEnergy,readoutNoise), N, cores):
    pool = mp.Pool(cores)
    data = (kbar,gridSize,speckleSize,detectorSize,chargeCloudSize,photonEnergy,readoutNoise)
    
    result = pool.starmap(DetectorImageBatch, [data for range(cores)])
    return result

In [ ]:
images = threadImages((0.1,[1024,1024],50,[32,32],2,9500,3),N,mp.cpu_count())

In [37]:
import h5py as h5

def saveThreadImages(images,title):
    img = [elt[i] for elt in images for i in range(np.shape(elt)[0])]
    h = h5.File(str(title)+'.hdf5','w')
    h.create_dataset("80", data = img)
    h.close()

In [ ]:
result, names = threading()

In [ ]:
def saveThreading(result,names,title)
    h = h5.File(str(title)+'.hdf5','w')
    for i,name in enumerate(names):
        for j in range(np.shape(result)[1]):
            h.create_dataset(name + str(j) + ".hdf5", data = result[i][j])
    h.close()